In [1]:
from __future__ import absolute_import, division, print_function

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras import layers

tf.VERSION

W0512 02:20:37.906744 25216 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


'1.13.1'

In [2]:
data_root = tf.keras.utils.get_file(
  'flower_photos','https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
   untar=True)


In [3]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
image_data = image_generator.flow_from_directory(str(data_root))


Found 3670 images belonging to 5 classes.


In [4]:
for image_batch,label_batch in image_data:
  print("Image batch shape: ", image_batch.shape)
  print("Labe batch shape: ", label_batch.shape)
  break


Image batch shape:  (32, 256, 256, 3)
Labe batch shape:  (32, 5)


In [40]:
import tensorflow.keras.backend as K
sess = K.get_session()
init = tf.global_variables_initializer()
sess.run(init)


In [41]:
feature_extractor_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/2" #@param {type:"string"}


In [42]:
# def feature_extractor(x):
#   feature_extractor_module = hub.Module(feature_extractor_url)
#   return feature_extractor_module(x)
module = hub.Module(feature_extractor_url)
IMAGE_SIZE = hub.get_expected_image_size(module)


In [43]:
image_data = image_generator.flow_from_directory(str(data_root), target_size=IMAGE_SIZE)
for image_batch,label_batch in image_data:
  print("Image batch shape: ", image_batch.shape)
  print("Labe batch shape: ", label_batch.shape)
  break


Found 3670 images belonging to 5 classes.
Image batch shape:  (32, 224, 224, 3)
Labe batch shape:  (32, 5)


In [44]:
features_extractor_layer = layers.Lambda(feature_extractor, input_shape=IMAGE_SIZE+[3])


In [45]:
print(image_batch.shape)

(32, 224, 224, 3)


In [36]:
# features = module(image_batch)
# print(features.shape)

In [46]:
  outputs = module(dict(images=image_batch), signature="image_feature_vector",
                   as_dict=True)
  features = outputs["default"]


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0512 03:16:12.599001 25216 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [49]:
print(features.shape)


(32, 1280)


In [54]:
import numpy as np

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  sess.run(tf.tables_initializer())

  result = sess.run(features)
  feature_vector = np.squeeze(result)

In [55]:
print(feature_vector.shape)

(32, 1280)
